In [3]:
import cirq
import numpy as np

def qft(qubits):
    circuit = cirq.Circuit()
    for i in range(len(qubits)):
        circuit.append(cirq.H(qubits[i]))
        for j in range(i+1, len(qubits)):
            circuit.append(cirq.CZ(qubits[i], qubits[j])**((2*np.pi)/2**(j-i)))
    for i in range(len(qubits)//2):
        circuit.append(cirq.SWAP(qubits[i], qubits[len(qubits)-1-i]))
    return circuit

def aqft(qubits, max_rotations):
    circuit = cirq.Circuit()
    for i in range(len(qubits)):
        circuit.append(cirq.H(qubits[i]))
        for j in range(i+1, min(i+max_rotations+1, len(qubits))):
            circuit.append(cirq.CZ(qubits[i], qubits[j])**((2*np.pi)/2**(j-i)))
    for i in range(len(qubits)//2):
        circuit.append(cirq.SWAP(qubits[i], qubits[len(qubits)-1-i]))
    return circuit

def phase_sum(qubits_A, qubits_B):
    circuit = cirq.Circuit()
    for i, q_A in enumerate(qubits_A):
        for j, q_B in enumerate(qubits_B):
            if j <= i:
                angle = np.pi/2**(i-j)
                circuit.append(cirq.CZ(q_B, q_A)**angle)
    return circuit

def draper_adder(qubits_A, qubits_B):
    circuit = cirq.Circuit()

    # QFT
    circuit.append(qft(qubits_A))

    # Suma en el dominio de la fase
    circuit.append(phase_sum(qubits_A, qubits_B))

    # AQFT
    circuit.append(aqft(qubits_A, 2))

    # QFT inversa
    circuit.append(qft(qubits_A)**-1)

    return circuit

qubits_A = cirq.LineQubit.range(4)
qubits_B = cirq.LineQubit.range(4, 8)

circuit = draper_adder(qubits_A, qubits_B)

circuit.append(cirq.measure(*qubits_A))
circuit.append(cirq.measure(*qubits_B))

simulator = cirq.Simulator()
result = simulator.simulate(circuit)

print(result)


# La medición 0011 0000 significa que el estado de los qubits A es 0011 (en decimal es 3) y
# el estado de los qubits B es 0000 (en decimal es 0). El resultado de la suma de 3 y 0 es 3.

# Los vectores de salida para cada qubit indican el estado del qubit después de aplicar el circuito. 
# En el caso de los qubits 0 y 1, los vectores de salida son |0⟩, lo que significa que los qubits están en el estado
# de superposición. Los qubits 2 y 3 están en el estado |1⟩, lo que significa que se midieron en el estado |1⟩. 
# Los qubits 4, 5, 6 y 7 están en el estado |0⟩, lo que significa que no cambiaron después de aplicar el circuito.

# El último resultado |⟩ representa el estado global del sistema después de aplicar el circuito, lo que significa que
# todos los qubits están en el estado de superposición.


measurements: q(0),q(1),q(2),q(3)=0011 q(4),q(5),q(6),q(7)=0000

qubits: (cirq.LineQubit(0),)
output vector: |0⟩

qubits: (cirq.LineQubit(1),)
output vector: |0⟩

qubits: (cirq.LineQubit(2),)
output vector: |1⟩

qubits: (cirq.LineQubit(3),)
output vector: |1⟩

qubits: (cirq.LineQubit(4),)
output vector: |0⟩

qubits: (cirq.LineQubit(5),)
output vector: |0⟩

qubits: (cirq.LineQubit(6),)
output vector: |0⟩

qubits: (cirq.LineQubit(7),)
output vector: |0⟩

phase:
output vector: |⟩
